In [87]:
from folktables import ACSDataSource
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split


In [5]:
data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')
ca_data = data_source.get_data(states=["CA"], download=True)

In [85]:
features = ["AGEP","SCHL","SEX","RAC1P","JWTR","POVPIP","JWMNP"]

ca_data_sub = ca_data[features].copy()
ca_data_sub.loc[np.isnan(ca_data_sub['SCHL']),'SCHL'] = 0
ca_data_sub.loc[np.isnan(ca_data_sub['JWTR']),'JWTR'] = 0

ca_data_sub.loc[ca_data_sub['SCHL'] <= 15,'SCHL'] = 0
ca_data_sub.loc[np.logical_and(ca_data_sub['SCHL'] <= 19, ca_data_sub['SCHL'] > 15),'SCHL'] = 1
ca_data_sub.loc[np.logical_or(ca_data_sub['SCHL'] == 20, ca_data_sub['SCHL'] == 21),'SCHL'] = 2
ca_data_sub.loc[ca_data_sub['SCHL'] > 21,'SCHL'] = 3

ca_data_sub['SCHL'] = ca_data_sub['SCHL'].astype('category')
ca_data_sub['SEX'] = ca_data_sub['SEX'].astype('category')
ca_data_sub['RAC1P'] = ca_data_sub['RAC1P'].astype('category')
ca_data_sub['JWTR'] = ca_data_sub['JWTR'].astype('category')


ca_data_sub = ca_data_sub[~ca_data_sub.isnull().any(axis=1)]

In [88]:
Y = ca_data_sub['JWMNP']
X = pd.get_dummies(ca_data_sub)
X = X.drop(columns = ['JWMNP'])
X.head()

x_train, x_calib, y_train, y_calib = train_test_split(X, Y, test_size=math.floor(X.shape[0]/2), random_state=42)

reg = LinearRegression().fit(x_train, y_train)
scores_calib = np.abs(reg.predict(x_calib) - y_calib)


